<a href="https://colab.research.google.com/github/futurexskill/ml-model-deployment/blob/main/pytorch_create_save.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

import pandas as pd
import numpy as np

In [2]:
dataset = pd.read_csv('https://raw.githubusercontent.com/futurexskill/ml-model-deployment/main/storepurchasedata_large.csv')

In [3]:
dataset.describe()

,Age,Salary,Purchased
count,1554.000000,1554.000000,1554.000000
mean,44.296010,57042.471042,0.694981
std,17.462458,21209.244800,0.460564
min,18.000000,20000.000000,0.000000
25%,27.000000,46000.000000,0.000000
50%,43.000000,60000.000000,1.000000
75%,62.000000,66000.000000,1.000000
max,69.000000,96000.000000,1.000000


In [4]:
dataset.head()

,Age,Salary,Purchased
0,18,20000,0
1,19,22000,0
2,20,24000,0
3,21,28000,0
4,22,60000,1


In [5]:
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:,-1].values

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =.20,random_state=0)

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [8]:
Xtrain_ = torch.from_numpy(X_train).float()
Xtest_ = torch.from_numpy(X_test).float()

In [9]:
Xtrain_

tensor([[-0.0935,  0.2215],
        [ 1.2217, -0.5342],
        [-1.0655,  0.4104],
        ...,
        [ 1.2789,  0.4104],
        [-0.9512,  0.2215],
        [-1.2943, -1.4316]])

In [10]:
ytrain_ = torch.from_numpy(y_train)
ytest_ = torch.from_numpy(y_test)

In [11]:
ytrain_

tensor([1, 1, 1,  ..., 1, 1, 0])

In [12]:
Xtrain_.shape, ytrain_.shape

(torch.Size([1243, 2]), torch.Size([1243]))

In [13]:
Xtest_.shape, ytest_.shape

(torch.Size([311, 2]), torch.Size([311]))

In [14]:
input_size=2
output_size=2
hidden_size=10

In [15]:
class Net(nn.Module):
   def __init__(self):
       super(Net, self).__init__()
       self.fc1 = torch.nn.Linear(input_size, hidden_size)
       self.fc2 = torch.nn.Linear(hidden_size, hidden_size)
       self.fc3 = torch.nn.Linear(hidden_size, output_size)


   def forward(self, X):
       X = torch.relu((self.fc1(X)))
       X = torch.relu((self.fc2(X)))
       X = self.fc3(X)

       return F.log_softmax(X,dim=1)

In [16]:
model = Net()



In [17]:
import torch.optim as optim
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.NLLLoss()

In [18]:
epochs = 100


In [19]:
for epoch in range(epochs):
  optimizer.zero_grad()
  Ypred = model(Xtrain_)
  loss = loss_fn(Ypred,  ytrain_)
  loss.backward()
  optimizer.step()
  print('Epoch',epoch, 'loss',loss.item())

Epoch 0 loss 0.7977757453918457
Epoch 1 loss 0.7583175897598267
Epoch 2 loss 0.7241986393928528
Epoch 3 loss 0.6962988972663879
Epoch 4 loss 0.67305988073349
Epoch 5 loss 0.6516295671463013
Epoch 6 loss 0.6316030621528625
Epoch 7 loss 0.612375020980835
Epoch 8 loss 0.5939561128616333
Epoch 9 loss 0.5765038728713989
Epoch 10 loss 0.558681070804596
Epoch 11 loss 0.5401478409767151
Epoch 12 loss 0.5201534628868103
Epoch 13 loss 0.49849650263786316
Epoch 14 loss 0.4755703806877136
Epoch 15 loss 0.45163261890411377
Epoch 16 loss 0.42660951614379883
Epoch 17 loss 0.4015096127986908
Epoch 18 loss 0.37676313519477844
Epoch 19 loss 0.3525939881801605
Epoch 20 loss 0.3294699192047119
Epoch 21 loss 0.30786263942718506
Epoch 22 loss 0.28793075680732727
Epoch 23 loss 0.26976385712623596
Epoch 24 loss 0.25348860025405884
Epoch 25 loss 0.23911052942276
Epoch 26 loss 0.2264137864112854
Epoch 27 loss 0.21513520181179047
Epoch 28 loss 0.20509260892868042
Epoch 29 loss 0.19615420699119568
Epoch 30 loss 0

In [20]:
list(model.parameters())

[Parameter containing:
 tensor([[-3.8818e-04, -8.0533e-03],
         [ 1.4851e-01, -4.0265e-03],
         [-8.0533e-02, -1.1202e+00],
         [ 1.1377e+00, -1.4474e-01],
         [ 3.7951e-01,  3.5863e-01],
         [ 3.2903e-01,  8.3079e-01],
         [-1.2451e+00,  2.6485e-03],
         [ 4.9578e-01,  2.7482e-01],
         [-5.7161e-01, -9.1517e-01],
         [-5.3357e-01, -8.6224e-01]], requires_grad=True),
 Parameter containing:
 tensor([ 0.6489, -0.4428,  0.3464,  0.0952,  0.0265,  0.3300,  0.9591,  0.7827,
          0.1555, -0.0011], requires_grad=True),
 Parameter containing:
 tensor([[ 1.1917e-01,  2.6128e-02,  3.2271e-01,  7.4611e-01,  9.6748e-02,
           1.9217e-01, -6.9655e-01,  4.6731e-01, -4.0125e-01,  1.1068e-01],
         [-2.4070e-01, -1.7209e-01, -4.5660e-02, -6.7476e-02, -1.1687e-01,
          -3.0379e-01, -4.1392e-02,  1.1405e-01, -1.7160e-01,  1.2158e-01],
         [-1.7269e-01, -7.9161e-03,  3.1154e-01, -1.7067e-01, -1.9979e-01,
          -7.2368e-02,  1.3492e-

In [21]:
torch.from_numpy(sc.transform(np.array([[40,20000]]))).float()

tensor([[-0.2650, -1.7622]])

In [22]:
y_cust_20_40000 = model(torch.from_numpy(sc.transform(np.array([[40,20000]]))).float())
y_cust_20_40000

tensor([[-0.0068, -4.9884]], grad_fn=<LogSoftmaxBackward>)

In [23]:
_, predicted_20_40000 = torch.max(y_cust_20_40000.data,-1)
predicted_20_40000

tensor([0])

In [24]:
y_cust_42_50000 = model(torch.from_numpy(sc.transform(np.array([[42,50000]]))).float())
y_cust_42_50000

tensor([[-0.7735, -0.6188]], grad_fn=<LogSoftmaxBackward>)

In [25]:
_, predicted_42_50000 = torch.max(y_cust_42_50000.data,-1)
predicted_42_50000

tensor([1])

In [26]:
torch.save(model,'customer_buy.pt')

In [27]:
!ls

customer_buy.pt  sample_data


In [28]:
restored_model = torch.load('customer_buy.pt')

In [29]:
y_cust_20_40000 = restored_model(torch.from_numpy(sc.transform(np.array([[40,20000]]))).float())
y_cust_20_40000

tensor([[-0.0068, -4.9884]], grad_fn=<LogSoftmaxBackward>)

In [30]:
_, predicted_20_40000 = torch.max(y_cust_20_40000.data,-1)
predicted_20_40000

tensor([0])

In [31]:
model.state_dict()

OrderedDict([('fc1.weight', tensor([[-3.8818e-04, -8.0533e-03],
                      [ 1.4851e-01, -4.0265e-03],
                      [-8.0533e-02, -1.1202e+00],
                      [ 1.1377e+00, -1.4474e-01],
                      [ 3.7951e-01,  3.5863e-01],
                      [ 3.2903e-01,  8.3079e-01],
                      [-1.2451e+00,  2.6485e-03],
                      [ 4.9578e-01,  2.7482e-01],
                      [-5.7161e-01, -9.1517e-01],
                      [-5.3357e-01, -8.6224e-01]])),
             ('fc1.bias',
              tensor([ 0.6489, -0.4428,  0.3464,  0.0952,  0.0265,  0.3300,  0.9591,  0.7827,
                       0.1555, -0.0011])),
             ('fc2.weight',
              tensor([[ 1.1917e-01,  2.6128e-02,  3.2271e-01,  7.4611e-01,  9.6748e-02,
                        1.9217e-01, -6.9655e-01,  4.6731e-01, -4.0125e-01,  1.1068e-01],
                      [-2.4070e-01, -1.7209e-01, -4.5660e-02, -6.7476e-02, -1.1687e-01,
                       -3.0

In [32]:
torch.save(model.state_dict(),'customer_buy_state_dict')

In [33]:
!ls

customer_buy.pt  customer_buy_state_dict  sample_data


In [35]:
new_predictor = Net()

In [36]:
y_cust_20_40000 = new_predictor(torch.from_numpy(sc.transform(np.array([[40,20000]]))).float())
y_cust_20_40000

tensor([[-0.6662, -0.7208]], grad_fn=<LogSoftmaxBackward>)

In [34]:
!zip -r customer_buy_state_dict.zip customer_buy_state_dict

  adding: customer_buy_state_dict (deflated 47%)


In [37]:
!ls

customer_buy.pt		 customer_buy_state_dict.zip
customer_buy_state_dict  sample_data


In [38]:
from google.colab import files


In [39]:
files.download('customer_buy_state_dict.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>